In [1]:
# Set number of GPUs
num_gpus = 1   #defaults to 1 if one-GPU or one-CPU. If 4 GPUs, set to 4.

# Set height (y-axis length) and width (x-axis length) to train model on
img_height, img_width = (256,256)  #Default to (256,266), use (None,None) if you do not want to resize imgs

In [2]:
# Import all the necessary libraries
import os
import datetime
import glob
import random
import sys

import matplotlib.pyplot as plt
import skimage.io                                     #Used for imshow function
import skimage.transform                              #Used for resize function
from skimage.morphology import label                  #Used for Run-Length-Encoding RLE to create final submission

import numpy as np
import pandas as pd

import keras
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Conv2DTranspose
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.models import load_model, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.merge import add, concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import multi_gpu_model, plot_model
from keras import backend as K
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split


print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Skimage      :', skimage.__version__)
print('Scikit-learn :', sklearn.__version__)
print('Keras        :', keras.__version__)
print('Tensorflow   :', tf.__version__)

/Users/wajdiahmed/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/wajdiahmed/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Python       : 3.6.3 |Anaconda custom (64-bit)| (default, Oct  6 2017, 12:04:38) 
Numpy        : 1.14.2
Skimage      : 0.13.1
Scikit-learn : 0.19.1
Keras        : 2.1.6
Tensorflow   : 1.10.1


In [3]:
# Set seed values
seed = 42
random.seed = seed
np.random.seed(seed=seed)

In [7]:
# Have a look at our data folder
topDir = '/Users/wajdiahmed/Kaggle' #defaults to '/kaggle' in kaggle kernels, different if on own system e.g. '/home/user/kaggle/dsbowl'
os.chdir(topDir)    #changes our python working directory to the top directory of our kaggle files
print(os.listdir(os.path.join(topDir, '')))  #see what's in the input folder (where data is in)

['.DS_Store', 'TCGA Data', 'stage1_solution.csv', 'NewData', 'stage1_test copy', 'extra_data', 'stage2_test_final', 'Thes_Antonyms_a-z.csv', 'stage1_test', 'stage1_sample_submission.csv', 'stage1_train', 'Square_cut_v1', 'Square_cut', 'mask.psd', 'stage1_train_labels.csv', 'mask.png', 'Single_Cut', 'image2.png', 'stage2_sample_submission_final.csv', 'image.png']


In [8]:
train_path = os.path.join(topDir, 'Square_cut')  #path to training data file/folder
test_path = os.path.join(topDir, 'stage1_test')   #path to test data file/folder

In [9]:
%%time
# Get training data
def get_X_data(path, output_shape=(None, None)):
    '''
    Loads images from path/{id}/images/{id}.png into a numpy array
    '''
    img_paths = ['{0}/{1}/images/{1}.png'.format(path, id) for id in os.listdir(path)]
    X_data = np.array([skimage.transform.resize(skimage.io.imread(path)[:,:,:3], output_shape=output_shape, mode='constant', preserve_range=True) for path in img_paths], dtype=np.uint8)  #take only 3 channels/bands
    
    return X_data
X_train = get_X_data(train_path, output_shape=(img_height,img_width))
print(X_train.shape, X_train.dtype)

NotADirectoryError: [Errno 20] Not a directory: '/Users/wajdiahmed/Kaggle/Square_cut/.DS_Store/images/.DS_Store.png'